# Spectral Analysis of Optimized Universal Pulses

In [1]:
import numpy as np
import QDYN
from QDYN.shutil import mkdir
import subprocess as sp
from os.path import join
from generate_zeta_universal import write_config
from QDYNTransmonLib.io import FullHamLevels
from collections import OrderedDict
from scipy.io import FortranFile
import re
from notebook_utils import bare_decomposition
import operator

In [2]:
wc = 6.2 # GHZ
w2 = 5.9 # GHz
w1 = 6.00 # GHz
wd = 5.9325 # GHz
root = './runs_zeta_detailed/w2_%dMHz_wc_%dMHz' % (w2*1000, wc*1000)
mkdir(root)

In [3]:
T = 50 # ns (arbitrary gate duration)
rf = join(root, 'analyze_spectral'); mkdir(rf)
nt = int(max(2000, 100 * T))
write_config(join(rf, 'config'), T, nt, wc, w2, wd=wd, gate="CPHASE", J_T='SM', prop_guess='T')
sp.call(['tm_en_gh', '--rwa', rf])
with open(join(rf, 'diagonalize.log'), 'w') as out_fh:
    sp.call(['tm_en_logical_eigenstates', '--all-eigenstates', '--ctrl', rf], stdout=out_fh)

In [4]:
rf

'./runs_zeta_detailed/w2_5900MHz_wc_6200MHz/analyze_spectral'

In [5]:
levels = FullHamLevels(join(rf, 'eigenvalues.dat'))

In [6]:
def get_eigenstates(eigenstates_dump):
    """Return list of eigenstates (as numpy array) from unformatted fortran 
    dump generated by tm_en_logical_eigenstates. We apply the sign
    convenction that the largest entry in each eigenstate should be
    positive"""
    eigenstates = []
    f = FortranFile(eigenstates_dump, 'r')
    vals = f.read_reals(dtype=np.float64)
    W = np.reshape(vals, (150, 150), order='F')
    for i in range(150):
        v = W[:,i]
        v *= np.sign(v[np.argmax(np.abs(v))])
        eigenstates.append(v)
    return eigenstates

In [7]:
def get_logical_levels(diagonalize_log):
    """Get the 0-based indices of the logical eigenstates from the
    captured stdout of tm_en_logical_eigenstates"""
    ind = {}
    with open(diagonalize_log) as in_fh:
        for line in in_fh:
            m = re.match(r'level\s+(?P<index>\d+) in logical subspace .*Ket\{(?P<state>[01]{2})0\}', line)
            if m:
                ind[m.group('state')] = int(m.group('index')) - 1
    return ind

In [8]:
logical_levels = get_logical_levels(join(rf, 'diagonalize.log'))

In [9]:
logical_levels

{'00': 94, '01': 91, '10': 97, '11': 93}

In [10]:
eigenstates = get_eigenstates(join(rf, 'eigenstates.dump'))

In [11]:
def levels_to_lab_frame(levels, eigenstates, wd):
    """Convert levels.E in place to contain the eigenenergies in the
    lab frame, instead of the rotating frame. This makes it easier to
    interpret negative frequencies in the RWA. The frequency of the rotating
    frame is `wd` in GHz. Note that after the transformation, the
    eigenenergies will no longer be ordered"""
    nq = 5; nc = 6
    for k in range(nq*nq*nc):
        for i in range(nq):
            for j in range(nq):
                for n in range(nc):
                    l = i * nq*nc + j * nc + n # zero-based
                    w_l = (i+j+n) * wd
                    levels.E[k] += w_l * np.abs(eigenstates[k][l])**2

In [12]:
levels_to_lab_frame(levels, eigenstates, wd)

In [13]:
def read_coupling_strength(ham_ctrl_diag_dat):
    """Return a dict that maps a tuple (k,l) of 0-based indices of two
    eigenstates to the (dimensionless) coupling strength of the transition
    between them. This reads the file 'ham_ctrl_diag.dat'
    generated by tm_en_logical_eigenstates that contains the control
    Hamiltonian transformed to the eigenframe of the drift Hamiltonian
    """
    table = OrderedDict()
    ks, ls = np.genfromtxt(ham_ctrl_diag_dat, unpack=True, usecols=(0,1), dtype=int)
    cs = np.genfromtxt(ham_ctrl_diag_dat, unpack=True, usecols=(2, ))
    for (k, l, c) in zip(ks, ls, cs):
        table[(k-1, l-1)] = c
    return table

In [14]:
def build_transition_table(levels, coupling_strength):
    """Return a dict that maps a tuple (k,l) of 0-based indices of two
    eigenstates to the transition energy, in GHz. Energies will be in the
    lab frame or the rotating frame, depending on which frame `levels` is in.
    Only transitions with a coupling strengh > 1e-3 will be included. Only 
    transitions from a lower to a higher level will be included, i.e.
    all transition energies will be positive
    """
    table = OrderedDict()
    i = levels.level_i; j = levels.level_j; n = levels.level_n
    for k in range(len(levels.level)):
        for l in range(len(levels.level)):
            if levels.E[k] > levels.E[l]:
                continue
            if (i[l] >= 3) or (j[l] >= 3) or (n[l] >= 3):
                continue
            try:
                if abs(coupling_strength[(k, l)]) > 1e-3:
                    table[(k,l)] = abs(levels.E[k] - levels.E[l])
            except KeyError:
                pass # implies zero coupling strength
    return table

In [15]:
coupling_strength = read_coupling_strength(join(rf, 'ham_ctrl_diag.dat'))

In [16]:
transition_table = build_transition_table(levels, coupling_strength)

In [17]:
len(transition_table)

122

In [18]:
def find_closest_transitions(transition_table, freq, n=5):
    """Return list of tuples (i, j, f) of the `n` transitions in the
    transition_table that are closest to `freq` (in GHz), where i and j are
    0-based indices of the eigenstates in the transition, and f is the
    transition frequency
    """
    # freq (GHz) is absolute
    ind = np.argsort(np.abs(np.array(transition_table.values()) - freq))[:n]
    transitions = [transition_table.keys()[i] for i in ind]
    return [(i, j, transition_table[(i,j)]) for (i,j) in transitions]

In [19]:
def print_closest_transitions(transition_table, coupling_strength, levels, freq, wd, n=5):
    """Print a human-readable list of the `n` closest transition,
    cf. `find_closest_transition`. However, `freq` is in MHz, not GHz, and
    is relative to `wd` (in GHz). The printout includes the dressed-level
    identification of the involved states and the coupling strength. 
    """
    # freq (MHz) is relative to wd, which is still in GHz
    closest_transitions = find_closest_transitions(transition_table, (freq*1e-3)+wd, n)
    i = levels.level_i; j = levels.level_j; n = levels.level_n
    for (k, l, f) in closest_transitions:
        c = abs(coupling_strength[(k, l)])
        if "%.2f" % c != '0.00':
            print("%d%d%d-%d%d%d_d: %.3f MHz (%.2f)" % (i[k], j[k], n[k], i[l], j[l], n[l], (f-wd)*1000, c))

## Transitions out of the logical subspace

In [20]:
def get_level_index(i, j, n):
    """Return the 0-based index of the eigenstate closest to the bare
    state ket{ijn}, which we identify as the dressed state ket{ijn}
    """
    for ind, (ii, jj, nn) in enumerate(zip(levels.level_i, levels.level_j, levels.level_n)):
        if (ii == i) and (jj == j) and (nn == n):
            return ind

In [21]:
def print_transition_info(ind1, ind2, levels, coupling_strength, eigenstates, wd, show_decomposition=True):
    if levels.E[ind1] > levels.E[ind2]:
        ind1, ind2 = ind2, ind1
    i = levels.level_i; j = levels.level_j; n = levels.level_n
    freq = levels.E[ind2] - levels.E[ind1] - wd
    c = coupling_strength[(ind1, ind2)]
    print "%7.2f MHz Transition %d%d%d_d -> %d%d%d_d" % (freq*1000, i[ind1], j[ind1], n[ind1], i[ind2], j[ind2], n[ind2])
    if show_decomposition:
        print "      %d%d%d_d = %s" % (i[ind1], j[ind1], n[ind1], bare_decomposition(eigenstates[ind1], 5, 6))
        print "      %d%d%d_d = %s" % (i[ind2], j[ind2], n[ind2], bare_decomposition(eigenstates[ind2], 5, 6))
    print "      transition amplitude: %.2f" % abs(c)

In [22]:
print "*** Driving between the logical states ***"
transition_info_indexes = [
    (logical_levels['00'], logical_levels['01']),
    (logical_levels['10'], logical_levels['11']),
    (logical_levels['00'], logical_levels['10']),
    (logical_levels['01'], logical_levels['11']),
]
for ind1, ind2 in transition_info_indexes:
    print_transition_info(ind1, ind2, levels, coupling_strength, eigenstates, wd)
print "*** Driving (dressed) qubit transition 0->1, while (dressed) cavity excited ***"
transition_info_indexes = [
    (get_level_index(0, 0, 1), get_level_index(0, 1, 1)),
    (get_level_index(1, 0, 1), get_level_index(1, 1, 1)),
    (get_level_index(0, 0, 1), get_level_index(1, 0, 1)),
    (get_level_index(0, 1, 1), get_level_index(1, 1, 1)),
]
for ind1, ind2 in transition_info_indexes:
    print_transition_info(ind1, ind2, levels, coupling_strength, eigenstates, wd)
print "*** Driving (dressed) qubit transition 1->2 ***"
transition_info_indexes = [
    (get_level_index(0, 1, 0), get_level_index(0, 2, 0)),
    (get_level_index(1, 0, 0), get_level_index(2, 0, 0)),
    (get_level_index(1, 1, 0), get_level_index(1, 2, 0)),
    (get_level_index(1, 1, 0), get_level_index(2, 1, 0)),
]
for ind1, ind2 in transition_info_indexes:
    print_transition_info(ind1, ind2, levels, coupling_strength, eigenstates, wd)
print "*** Driving the (dressed) cavity ***"
transition_info_indexes = [
    (get_level_index(0, 0, 0), get_level_index(0, 0, 1)),
    (get_level_index(0, 1, 0), get_level_index(0, 1, 1)),
    (get_level_index(1, 0, 0), get_level_index(1, 0, 1)),
    (get_level_index(1, 1, 0), get_level_index(1, 1, 1)),
]
for ind1, ind2 in transition_info_indexes:
    print_transition_info(ind1, ind2, levels, coupling_strength, eigenstates, wd)

*** Driving between the logical states ***
 -50.25 MHz Transition 000_d -> 010_d
      000_d = 100.0% {000}
      010_d = 5.9% {001} + 92.0% {010} + 2.1% {100}
      transition amplitude: 0.12
 -49.95 MHz Transition 100_d -> 110_d
      100_d = 5.7% {001} + 4.2% {010} + 90.1% {100}
      110_d = 0.7% {002} + 7.9% {011} + 0.6% {020} + 3.4% {101} + 86.0% {110} + 1.4% {200}
      transition amplitude: 0.07
  49.82 MHz Transition 000_d -> 100_d
      000_d = 100.0% {000}
      100_d = 5.7% {001} + 4.2% {010} + 90.1% {100}
      transition amplitude: 0.12
  50.11 MHz Transition 010_d -> 110_d
      010_d = 5.9% {001} + 92.0% {010} + 2.1% {100}
      110_d = 0.7% {002} + 7.9% {011} + 0.6% {020} + 3.4% {101} + 86.0% {110} + 1.4% {200}
      transition amplitude: 0.16
*** Driving (dressed) qubit transition 0->1, while (dressed) cavity excited ***
 -62.28 MHz Transition 001_d -> 011_d
      001_d = 88.3% {001} + 3.8% {010} + 7.8% {100}
      011_d = 10.2% {002} + 76.7% {011} + 2.0% {020} + 0.6%

In [23]:
def get_low_levels():
    """Return set of logical indices or level indices that can be reached
    immediately from the logical subspace"""
    low_levels = set(logical_levels.values())
    logical_indices = set(logical_levels.values())
    for (ind1, ind2) in coupling_strength.keys():
        if ind1 in logical_indices:
            low_levels.add(ind2)
        if ind2 in logical_indices:
            low_levels.add(ind1)
    return low_levels

In [24]:
for ind in get_low_levels():
    print "%d%d%d_d" % (levels.level_i[ind], levels.level_j[ind], levels.level_n[ind])

013_d
003_d
030_d
031_d
300_d
311_d
020_d
120_d
221_d
210_d
033_d
200_d
021_d
010_d
110_d
000_d
211_d
100_d
201_d
122_d
011_d
111_d
001_d
212_d
101_d
202_d
012_d
002_d
102_d


In [25]:
def print_dominant_transitions(subspace, exclude=None):
    """Print sorted list of transition within or out of the given subspace"""
    def sort_key(item):
        (ind1, ind2), c = item
        return -abs(c)
    sorted_coupling_strength = sorted(coupling_strength.items(), key=sort_key)
    printed = 0
    if exclude is None:
        exclude = set([])
    for ((ind1, ind2), c) in sorted_coupling_strength:
        if levels.E[ind1] > levels.E[ind2]:
            continue
        i = levels.level_i; j = levels.level_j; n = levels.level_n
        freq = abs((levels.E[ind2] - levels.E[ind1] - wd)) * 1000
        if ind1 in subspace or ind2 in subspace:
            if ind1 in exclude or ind2 in exclude:
                continue
            print_transition_info(ind1, ind2, levels, coupling_strength, eigenstates, wd, show_decomposition=False)
            printed += 1
            if abs(c) < 1e-2:
                return

In [26]:
print_dominant_transitions(subspace=set(logical_levels.values()))

 267.53 MHz Transition 110_d -> 111_d
      transition amplitude: 0.52
 282.28 MHz Transition 100_d -> 101_d
      transition amplitude: 0.50
 290.88 MHz Transition 010_d -> 011_d
      transition amplitude: 0.48
 302.92 MHz Transition 000_d -> 001_d
      transition amplitude: 0.47
  50.11 MHz Transition 010_d -> 110_d
      transition amplitude: 0.16
 -50.25 MHz Transition 000_d -> 010_d
      transition amplitude: 0.12
  49.82 MHz Transition 000_d -> 100_d
      transition amplitude: 0.12
-225.65 MHz Transition 100_d -> 200_d
      transition amplitude: 0.10
-347.32 MHz Transition 110_d -> 120_d
      transition amplitude: 0.09
-341.00 MHz Transition 010_d -> 020_d
      transition amplitude: 0.08
-219.73 MHz Transition 110_d -> 210_d
      transition amplitude: 0.08
 -49.95 MHz Transition 100_d -> 110_d
      transition amplitude: 0.07
-125.58 MHz Transition 010_d -> 200_d
      transition amplitude: 0.03
-441.06 MHz Transition 100_d -> 020_d
      transition amplitude: 0.01
 530.3

In [27]:
print_dominant_transitions(subspace=get_low_levels(), exclude=set(logical_levels.values()))

 288.01 MHz Transition 013_d -> 014_d
      transition amplitude: 0.97
 287.77 MHz Transition 033_d -> 034_d
      transition amplitude: 0.97
 299.37 MHz Transition 003_d -> 004_d
      transition amplitude: 0.95
 269.87 MHz Transition 122_d -> 123_d
      transition amplitude: 0.88
 269.23 MHz Transition 212_d -> 213_d
      transition amplitude: 0.88
 282.89 MHz Transition 102_d -> 103_d
      transition amplitude: 0.86
 281.62 MHz Transition 202_d -> 203_d
      transition amplitude: 0.85
 288.71 MHz Transition 012_d -> 013_d
      transition amplitude: 0.84
 286.76 MHz Transition 032_d -> 033_d
      transition amplitude: 0.83
 300.37 MHz Transition 002_d -> 003_d
      transition amplitude: 0.82
 268.87 MHz Transition 111_d -> 112_d
      transition amplitude: 0.73
 267.77 MHz Transition 121_d -> 122_d
      transition amplitude: 0.72
 267.82 MHz Transition 211_d -> 212_d
      transition amplitude: 0.72
 266.79 MHz Transition 221_d -> 222_d
      transition amplitude: 0.72
 267.9

## Peaks in optimized pulses 

### Hadamard (1)

In [28]:
print_closest_transitions(transition_table, coupling_strength, levels, 40, wd, n=3)

020-120_d: 43.787 MHz (0.15)
000-100_d: 49.823 MHz (0.12)
010-110_d: 50.115 MHz (0.16)


In [29]:
print_closest_transitions(transition_table, coupling_strength, levels, 80, wd, n=5)

302-222_d: 91.993 MHz (0.01)
110-201_d: 106.679 MHz (0.01)
301-221_d: 107.165 MHz (0.01)


In [30]:
print_closest_transitions(transition_table, coupling_strength, levels, -80, wd, n=5)

102-112_d: -78.468 MHz (0.07)
002-012_d: -74.195 MHz (0.11)
202-212_d: -73.203 MHz (0.10)
101-111_d: -64.706 MHz (0.07)


In [31]:
print_closest_transitions(transition_table, coupling_strength, levels, -100, wd, n=5)

102-112_d: -78.468 MHz (0.07)
010-200_d: -125.581 MHz (0.03)
002-012_d: -74.195 MHz (0.11)


In [32]:
print_closest_transitions(transition_table, coupling_strength, levels, -140, wd, n=5)

012-202_d: -141.740 MHz (0.03)
011-201_d: -134.089 MHz (0.03)
010-200_d: -125.581 MHz (0.03)
200-120_d: -171.627 MHz (0.02)


### Hadamard (2)

In [33]:
print_closest_transitions(transition_table, coupling_strength, levels, -20, wd, n=3)

022-122_d: 0.083 MHz (0.12)
200-210_d: -44.030 MHz (0.11)
012-112_d: 6.011 MHz (0.13)


In [34]:
print_closest_transitions(transition_table, coupling_strength, levels, -40, wd, n=3)

200-210_d: -44.030 MHz (0.11)
100-110_d: -49.954 MHz (0.07)
000-010_d: -50.245 MHz (0.12)


In [35]:
print_closest_transitions(transition_table, coupling_strength, levels, -160, wd, n=5)

200-120_d: -171.627 MHz (0.02)
012-202_d: -141.740 MHz (0.03)
011-201_d: -134.089 MHz (0.03)
201-121_d: -187.233 MHz (0.02)
010-200_d: -125.581 MHz (0.03)


### Phase (2)

In [36]:
print_closest_transitions(transition_table, coupling_strength, levels, -260, wd, n=5)

102-202_d: -226.220 MHz (0.10)
100-200_d: -225.650 MHz (0.10)
101-201_d: -225.556 MHz (0.10)
112-212_d: -220.955 MHz (0.08)
111-211_d: -219.907 MHz (0.08)


### BGATE

In [37]:
print_closest_transitions(transition_table, coupling_strength, levels, -160, wd, n=5)

200-120_d: -171.627 MHz (0.02)
012-202_d: -141.740 MHz (0.03)
011-201_d: -134.089 MHz (0.03)
201-121_d: -187.233 MHz (0.02)
010-200_d: -125.581 MHz (0.03)


In [38]:
print_closest_transitions(transition_table, coupling_strength, levels, -180, wd, n=5)

201-121_d: -187.233 MHz (0.02)
200-120_d: -171.627 MHz (0.02)
202-122_d: -201.436 MHz (0.02)
120-220_d: -215.829 MHz (0.11)
121-221_d: -215.959 MHz (0.10)


In [39]:
print_closest_transitions(transition_table, coupling_strength, levels, -280, wd, n=5)

102-202_d: -226.220 MHz (0.10)
100-200_d: -225.650 MHz (0.10)
101-201_d: -225.556 MHz (0.10)


In [40]:
print_closest_transitions(transition_table, coupling_strength, levels, -420, wd, n=5)

102-022_d: -427.740 MHz (0.01)
101-021_d: -433.512 MHz (0.01)
100-020_d: -441.064 MHz (0.01)
011-210_d: -460.494 MHz (0.01)
021-220_d: -461.875 MHz (0.01)


In [41]:
print_closest_transitions(transition_table, coupling_strength, levels, -540, wd, n=8)

003-202_d: -516.305 MHz (0.01)
023-222_d: -504.178 MHz (0.01)
013-212_d: -503.654 MHz (0.02)
002-201_d: -497.907 MHz (0.01)
